<br><br><br><br><br><h1 style="font-size:4em;color:#2467C0">Welcome to Week 3</h1><br><br><br>


<p style="line-height:31px;">This document provides a running example of completing the Week 3 assignment : </p>
<ul class="simple">
<li style="line-height:31px;">You can then copy paste the commands below in pySpark. </li>
<li style="line-height:31px;">To open pySpark, type pyspark and hit enter</li>
<br>
Finally, make sure that your working directory contains the data file(s) (.csv) for the fastest support. <br><br> We recommend workng in your home directory (type cd ~ in your terminal). Please run any scripts using your terminal for proper settings
<br>

<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">
<br><br><br>
To run these commands in Cloudera VM: first run the setup script: setupWeek3.sh</p>

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

<br><br>

<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Import Data
</p>

 

<br><br>
<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">First let us read the contents of the file w3_clustering.csv. The following commands read in the CSV file <br> in a table format. <br><br>


If you want to run this command on your machine: Note that you must change the path in the following <br>
command to 'w3_clustering.csv' to point to the location on your local machine.
</p>

</div>

<br><br><br><br>

In [2]:
sc = SparkContext()
sqlContext = SQLContext(sc)
data = sqlContext.read.load('Data_For_Clustering_3_Features.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

In [3]:
data.toPandas().head()

,userId,purchase_per_adclick,avg_session_duration,hit_rate
0,0,0.000000,2632.500000,0.105535
1,1,0.477273,3750.988095,0.134078
2,2,0.000000,3834.000000,0.095238
3,6,0.000000,4470.000000,0.105960
4,8,5.300000,2400.000000,0.100000


<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Let us display the number of lines in the data:

</p>

<br><br>

In [4]:
data.count()

1193

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

What does the data contain ?
</p>

 
<br><br>

In [5]:
data

DataFrame[userId: int, purchase_per_adclick: double, avg_session_duration: double, hit_rate: double]

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Statistics about the data:
</p>

 
<br><br>

In [6]:
data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
userId,1193,1194.1458507963118,694.8750700218611,0,2389
purchase_per_adclick,1193,0.7322594954872469,1.5638370519652285,0.0,25.0
avg_session_duration,1193,3185.4634869410183,1365.7021428549751,0.0,5880.0
hit_rate,1193,0.1109326830553271,0.029146254454120384,0.0,0.5


<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Drop all rows with NULL or NaN values

</p>

 
<br><br>

In [7]:
data = data.na.drop()

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Let us look at the column names:

</p>

 
 
<br><br>

In [8]:
data.columns

['userId', 'purchase_per_adclick', 'avg_session_duration', 'hit_rate']

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Use VectorAssembler to gather all the features:

</p>

 
<br><br>

In [9]:
featuresUsed = ['purchase_per_adclick', 'avg_session_duration', 'hit_rate']
assembler = VectorAssembler(inputCols=featuresUsed, outputCol="features_unscaled")
assembled = assembler.transform(data)

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Scale the features using StandardScaler:

</p>

 
<br><br>

In [10]:
scaler = StandardScaler(inputCol="features_unscaled", outputCol="features", withStd=True, withMean=True)
scalerModel = scaler.fit(assembled)
scaledData = scalerModel.transform(assembled)

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Select the features column make the data persist:

</p>

 
<br><br>

In [11]:
scaledData = scaledData.select("features")
scaledData.persist()

DataFrame[features: vector]

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

We can now perform KMeans clustering to generate 2 clusters:
</p>

 
<br><br>

In [12]:
kmeans = KMeans(k=4, seed=1)
model = kmeans.fit(scaledData)
transformed = model.transform(scaledData)

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Print the center of these two clusters:

</p>

 
<br><br>

In [13]:
centers = model.clusterCenters()
centers

[array([-0.21903156,  0.43540912, -0.0538909 ]),
 array([-0.32941065, -1.7305019 , -0.42341042]),
 array([-0.46824539, -2.23297675,  4.92736511]),
 array([2.3048937 , 0.09565116, 0.41877504])]

In [14]:
for i in range(4):
    print('Center of cluster number {}'.format(i+1))
    for j in range(3):
        print('{} = {:.4f}'.format(featuresUsed[j], centers[i][j]))
    print()

Center of cluster number 1
purchase_per_adclick = -0.2190
avg_session_duration = 0.4354
hit_rate = -0.0539

Center of cluster number 2
purchase_per_adclick = -0.3294
avg_session_duration = -1.7305
hit_rate = -0.4234

Center of cluster number 3
purchase_per_adclick = -0.4682
avg_session_duration = -2.2330
hit_rate = 4.9274

Center of cluster number 4
purchase_per_adclick = 2.3049
avg_session_duration = 0.0957
hit_rate = 0.4188



<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Analyze center of these two clusters:

</p>

 
<br><br>


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Each array denotes the center for a cluster:<br><br>
One Cluster is centered at   ... array([ 0.84174521,  0.51884657])<br>
Other Cluster is centered at   ... array([-0.79780796, -0.49176392])</p>

<br><br>

<p style="line-height:31px;"> First number (field1) in each array refers to scaled verson of the number of ad-clicks and the second number (field2) is the scaled version of the revenue per user.

Compare the 1st number of each cluster to see how differently users in each cluster behave when it comes to clicking ads.

Compare the 2nd number of each cluster to see how differently users in each cluster behave when it comes to buying stuff. 

</p><br><br>

<p style="line-height:31px;">In one cluster, in general, players click on ads much more often and spend more money on in-app purchases. Assuming that Eglence Inc. gets paid for showing ads and for hosting in-app purchase items, we can use this information to increase game's revenue by increasing the prices for ads we show to the frequent-clickers, and charge higher fees for hosting the in-app purchase items shown to the higher revenue generating buyers.</p>

<br><br>
<p style="line-height:31px;"> <b> Note: </b>  This analysis requires you to compare the cluster centers and find any ‘significant’ differences in the corresponding feature values of the  centers. The answer to this question will depend on the features you have chosen. <br><br> Some features help distinguish the clusters remarkably while others may not tell you much. At this point, if you don’t find clear distinguishing patterns, perhaps re-running the clustering model with different numbers of clusters and revising the features you picked would be a good idea. </p>

</div>
